<img src="img/pandora2d_logo.png" width="500" height="500">

# Pandora2D : a coregistration framework

# Usage of step and ROI (Region of Interest) parameters in the configuration file

#### Imports and external functions

In [ ]:
import io
import numpy as np
from pprint import pprint
from pathlib import Path
from IPython.display import Image, display
from copy import deepcopy

In [ ]:
def plot_state_machine(pandora2d_machine):
    """
    Show the schemes of step of Pandora2D Machine
    """
    stream = io.BytesIO()
    try:
        pandora2d_machine.get_graph().draw(stream, prog="dot", format="png")
        display(Image(stream.getvalue()))
    except:
        print(
            "It is not possible to show the graphic of the state machine. To solve it, please install graphviz on your system (apt-get install graphviz if operating in Linux) and install python package with pip insall graphviz"
        )

In [ ]:
from snippets.utils import *

# Pandora2D execution options with state machine

#### Imports of pandora2d

In [ ]:
# Load pandora2d imports
from pandora2d import run
from pandora2d.state_machine import Pandora2DMachine
from pandora2d.check_configuration import check_conf
from pandora2d.img_tools import create_datasets_from_inputs

#### Load and visualize input data 

Provide image path

In [ ]:
# Paths to left and right images
img_left_path = "data/left.tif"
img_right_path = "data/right.tif"

Convert input data to dataset

In [ ]:
input_config = {
    "left": {
        "img": img_left_path,
        "nodata": np.nan,
    },
    "right": {
        "img": img_right_path,
        "nodata": np.nan,
    },
    "col_disparity": {"init": 0, "range": 3},
    "row_disparity": {"init": 0, "range": 3},
}

create datasets

In [ ]:
image_datasets = create_datasets_from_inputs(input_config=input_config)

Visualize input data

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax0 = fig.add_subplot(1, 2, 1)
ax0.imshow(image_datasets.left["im"].data)
plt.title("Left image")
ax1 = fig.add_subplot(1, 2, 2)
ax1.imshow(image_datasets.right["im"].data)
plt.title("Right image")

#### Instantiate the machine

In [ ]:
pandora2d_machine = Pandora2DMachine()

#### Define pipeline configurations

Let’s define two configurations: one without refinement, the other with refinement.

In [ ]:
user_cfg = {
    "input": input_config,
    "pipeline": {
        "matching_cost": {
            "matching_cost_method": "zncc",
            "window_size": 7,
        },
        "disparity": {
            "disparity_method": "wta",
            "invalid_disparity": -9999,
        },
    },
    "output": {
        "path": "outputs/usage_dichotomy/base",
    },
}

Configuration with python dichotomy

In [ ]:
user_cfg_with_dicho_python = deepcopy(user_cfg)
user_cfg_with_dicho_python["pipeline"]["refinement"] = {
    "refinement_method": "dichotomy_python",
    "iterations": 1,
    "filter": {"method": "bicubic_python"},
}
user_cfg_with_dicho_python["output"] = {
        "path": "outputs/usage_dichotomy/one_iteration/python",
    }

In [ ]:
user_cfg_with_two_dicho_python = deepcopy(user_cfg)
user_cfg_with_two_dicho_python["pipeline"]["refinement"] = {
    "refinement_method": "dichotomy_python",
    "iterations": 2,
    "filter": {"method": "bicubic_python"},
}
user_cfg_with_two_dicho_python["output"] = {
        "path": "outputs/usage_dichotomy/two_iterations/python",
    }

Configuration with C++ dichotomy

In [ ]:
user_cfg_with_dicho_cpp = deepcopy(user_cfg)
user_cfg_with_dicho_cpp["pipeline"]["refinement"] = {
    "refinement_method": "dichotomy",
    "iterations": 1,
    "filter": {"method": "bicubic"},
}
user_cfg_with_dicho_cpp["output"] = {
        "path": "outputs/usage_dichotomy/one_iteration/cpp",
    }

In [ ]:
user_cfg_with_two_dicho_cpp = deepcopy(user_cfg)
user_cfg_with_two_dicho_cpp["pipeline"]["refinement"] = {
    "refinement_method": "dichotomy",
    "iterations": 2,
    "filter": {"method": "bicubic"},
}
user_cfg_with_two_dicho_cpp["output"] = {
        "path": "outputs/usage_dichotomy/two_iterations/cpp",
    }

#### Check the user configurations

In [ ]:
cfg = check_conf(user_cfg, pandora2d_machine)
pprint(cfg)

In [ ]:
cfg_with_dicho_python = check_conf(user_cfg_with_dicho_python, pandora2d_machine)
pprint(cfg_with_dicho_python)

In [ ]:
cfg_with_two_dicho_python = check_conf(user_cfg_with_two_dicho_python, pandora2d_machine)
pprint(cfg_with_two_dicho_python)

In [ ]:
cfg_with_dicho_cpp = check_conf(user_cfg_with_dicho_cpp, pandora2d_machine)
pprint(cfg_with_dicho_cpp)

In [ ]:
cfg_with_two_dicho_cpp = check_conf(user_cfg_with_two_dicho_cpp, pandora2d_machine)
pprint(cfg_with_two_dicho_cpp)

#### Execute the pipelines

In [ ]:
%%time
dataset, _ = run(pandora2d_machine, image_datasets.left, image_datasets.right, cfg)

In [ ]:
%%time
dataset_with_dicho_python, _ = run(pandora2d_machine, image_datasets.left, image_datasets.right, cfg_with_dicho_python)

In [ ]:
%%time
dataset_with_two_dicho_python, _ = run(
    pandora2d_machine, image_datasets.left, image_datasets.right, cfg_with_two_dicho_python
)

In [ ]:
%%time
dataset_with_dicho_cpp, _ = run(pandora2d_machine, image_datasets.left, image_datasets.right, cfg_with_dicho_cpp)

In [ ]:
%%time
dataset_with_two_dicho_cpp, _ = run(
    pandora2d_machine, image_datasets.left, image_datasets.right, cfg_with_two_dicho_cpp
)

# Visualize results

## Define ploting function

We will plot a figure with two maps: on left the disparity map of row and or right the disparity map of columns.
Disparities' colors will range from blue to red, disparities under minimum disparity will be green and disparies above maximum disparity will be yellow.

In [ ]:
from matplotlib import colors

plt.rcParams["figure.figsize"] = (20, 8)
min_max_disp_col = [input_config["col_disparity"]["init"] - input_config["col_disparity"]["range"], 
                    input_config["col_disparity"]["init"] + input_config["col_disparity"]["range"]]

min_max_disp_row = [input_config["row_disparity"]["init"] - input_config["row_disparity"]["range"], 
                    input_config["row_disparity"]["init"] + input_config["row_disparity"]["range"]]

def plot_result(
    dataset,
    dmin=min(min_max_disp_row[0], min_max_disp_col[0]),
    dmax=max(min_max_disp_row[1], min_max_disp_col[1]),
    palette=plt.cm.bwr.with_extremes(over="y", under="g", bad="k"),
    title="Without refinement.",
    colorbarlabel="Disparity",
):
    norm = colors.Normalize(vmin=dmin, vmax=dmax)
    fig, axs = plt.subplots(1, 2)
    fig.suptitle(title, fontsize=20)
    ax1, ax2 = axs
    ax1.set_title("Row disparity map")
    im1 = ax1.imshow(dataset["row_map"].data, cmap=palette, norm=norm)
    ax2.set_title("Columns disparity map")
    im2 = ax2.imshow(dataset["col_map"].data, cmap=palette, norm=norm)
    fig.colorbar(im1, ax=axs, orientation="horizontal", extend="both", shrink=0.75, aspect=50, label=colorbarlabel)

## Visualize resulting disparity maps

#### Without refinement

In [ ]:
plot_result(dataset)

#### Dichotomy python

In [ ]:
plot_result(dataset_with_dicho_python, title="With one iteration dichotomy python.")

In [ ]:
plot_result(dataset_with_two_dicho_python, title="With two iterations dichotomy python.")

#### Dichotomy cpp

In [ ]:
plot_result(dataset_with_dicho_cpp, title="With one iteration dichotomy cpp.")

In [ ]:
plot_result(dataset_with_two_dicho_cpp, title="With two iterations dichotomy cpp.")

## Difference between refined and not refined

Let’s see the subpixellic delta.

#### Dichotomy python

In [ ]:
plot_result(
    dataset - dataset_with_dicho_python,
    -1,
    1,
    plt.cm.Set1.with_extremes(over="y", under="g", bad="k"),
    title="Difference between refined with one iteration of dichotomy python and not refined.",
    colorbarlabel="Disparity difference",
)

In [ ]:
plot_result(
    dataset - dataset_with_two_dicho_python,
    -1,
    1,
    plt.cm.Set1.with_extremes(over="y", under="g", bad="k"),
    title="Difference between refined with two iterations of dichotomy python and not refined.",
    colorbarlabel="Disparity difference",
)

#### Dichotomy cpp

In [ ]:
plot_result(
    dataset - dataset_with_dicho_cpp,
    -1,
    1,
    plt.cm.Set1.with_extremes(over="y", under="g", bad="k"),
    title="Difference between refined with one iteration of dichotomy cpp and not refined.",
    colorbarlabel="Disparity difference",
)

In [ ]:
plot_result(
    dataset - dataset_with_two_dicho_cpp,
    -1,
    1,
    plt.cm.Set1.with_extremes(over="y", under="g", bad="k"),
    title="Difference between refined with two iterations of dichotomy cpp and not refined.",
    colorbarlabel="Disparity difference",
)